Task 1. Make a dataset

Task 2. Create a forecasting BigQuery machine learning model
Create the first machine learning model to predict the trip duration for bike trips.
The features of this model must incorporate the starting station name, the hour the trip started, the weekday of the trip, and the address of the start station labelled as location. You must use Training Year data only to train this model.

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
-- starting station name, the hour the trip started, the weekday of the trip, and the address of the start station labelled as location
%%bigquery
CREATE OR REPLACE MODEL ML_dataset.trained_model
OPTIONS (
  model_type='linear_reg',
  labels=['duration_minutes']
)
AS

SELECT
  start_station_name, 
  EXTRACT(HOUR FROM start_time) as hour,
  EXTRACT(dayofweek FROM start_time) as weekday,
  address location,
  duration_minutes
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
JOIN `bigquery-public-data.austin_bikeshare.bikeshare_stations`
ON station_id=station_id
WHERE start_time BETWEEN TIMESTAMP('2016-01-01') AND TIMESTAMP('2017-01-01')
-- LIMIT 10;

Task 3. Create the second machine learning model
Create the second machine learning model to predict the trip duration for bike trips.
The features of this model must incorporate the starting station name, the bike share subscriber type and the start time for the trip. You must also use Training Year data only to train this model.

In [ ]:
-- The features of this model must incorporate the starting station name, the bike share subscriber type and the start time for the trip. You must also use 2016 data only to train this model.
%%bigquery
CREATE OR REPLACE MODEL ML_dataset.trained_model2
OPTIONS (
  model_type='linear_reg',
  labels=['duration_minutes']
)
AS

SELECT
  start_station_name, 
  subscriber_type,
  EXTRACT(HOUR FROM start_time) AS start_hour,
  duration_minutes
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
-- JOIN `bigquery-public-data.austin_bikeshare.bikeshare_stations`
-- ON station_id=station_id
WHERE EXTRACT(YEAR FROM start_time) = 2016
-- LIMIT 10;

Task 4. Evaluate the two machine learning models

In [ ]:
%%bigquery
SELECT 
  SQRT(mean_squared_error) AS rmse,
  mean_absolute_error

FROM ML.EVALUATE(MODEL `qwiklabs-gcp-04-6b085ce229df.ML_dataset.trained_model`, (

  SELECT
    start_station_name, 
    EXTRACT(HOUR FROM start_time) as hour,
    EXTRACT(dayofweek FROM start_time) as weekday,
    address location,
    duration_minutes
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  JOIN `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  ON station_id=station_id
  WHERE start_time BETWEEN TIMESTAMP('2019-01-01') AND TIMESTAMP('2020-01-01')
))

In [ ]:
%%bigquery
SELECT 
  SQRT(mean_squared_error) AS rmse,
  mean_absolute_error

FROM ML.EVALUATE(MODEL `qwiklabs-gcp-04-6b085ce229df.ML_dataset.trained_model2`, (

  SELECT
    start_station_name, 
    subscriber_type,
    start_time,
    duration_minutes
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  -- JOIN `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  -- ON station_id=station_id
  WHERE start_time BETWEEN TIMESTAMP('2019-01-01') AND TIMESTAMP('2020-01-01')
))

Task 5. Use the subscriber type machine learning model to predict average trip durations

In [ ]:
%%bigquery
WITH busiest_station AS (
    SELECT
      start_station_id
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
    WHERE EXTRACT(YEAR FROM start_time) = 2019
    GROUP BY start_station_id
    ORDER BY COUNT(*) DESC
    LIMIT 10
  )

SELECT 
  AVG(predicted_duration_minutes) as predicted_average_trip_length

FROM ML.PREDICT(MODEL `qwiklabs-gcp-04-6b085ce229df.ML_dataset.trained_model2`, (
 
  SELECT
    trips.start_station_name, 
    trips.subscriber_type,
    EXTRACT(HOUR FROM start_time) AS start_hour,
    trips.duration_minutes
  FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` trips
  JOIN 
    busiest_station ON trips.start_station_id = busiest_station.start_station_id
  WHERE 
    EXTRACT(YEAR FROM trips.start_time) = 2019
    AND trips.subscriber_type = 'Single Trip'
  ))